In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This notebook demonstrates the EDA/feature engineering/prediction on the raw accelerator raw data
Dataset license: https://www.cis.fordham.edu/wisdm/dataset.php

The data is collected from the activities of 36 users
fs = 20Hz

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

In [3]:
# import the data : we use raw data only
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('/kaggle/input/wisdm/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header = None, names = columns)

In [4]:
df.head()

In [5]:
df.shape

In [6]:
# check the missing values
df.isnull().sum()

In [7]:
# df.isna().sum()/len(df)

In [8]:
# There's only 1 missing value in z axis, so we can drop it
df=df.dropna()

In [9]:
# investigate the data types
df.info()

In [10]:
df.isnull().sum()

There are two non-numeric features: activity and z-axis
We should encode these features or convert them to numeric data type

In [11]:
df['activity'].unique()

'activity' column needs to be encoded/mapped from string to a number
z-axis has "object" datatype, ending with ";"
We need to remove ";" sign and convert it to float.
We can use RegEx to extract the pattern of float number and then, convert the datatype


In [12]:
import re

#pattern = re.compile(r"\d+\.\d+")
#df['z-axis'] = df['z-axis'].apply(lambda x: re.findall(pattern,str(x)))

In [13]:
df.head()

In [14]:
#df.isnull().sum()

In [15]:
df['z-axis'] = df['z-axis'].str.replace(';', '')
df['z-axis'] = df['z-axis'].apply(lambda x:float(x))
df.isnull().sum()

In [16]:
df.head()

In [17]:
# check the row without valid timestamp
count = len([x for x in df['timestamp'] if x == 0])
print(count/len(df))

In [18]:
# 1.1% of data row do not have timestamp --> drop them
df = df[df['timestamp']!=0]
df.shape

In [19]:
# count the number of rows per user
df.groupby('user').size()

In [20]:
df.groupby('user')['activity'].describe()

In [21]:
# sorting data by 'user' and 'timestamp'
df = df.sort_values(by = ['user', 'timestamp'], ignore_index=True)

In [22]:
# There are 6 types of activities
# We need to investigate the number of samples per each activity
sns.set_style("whitegrid")
ax = sns.countplot(x='activity',data=df)
for container in ax.containers:
    ax.bar_label(container)

In [23]:
# visualize the activity per user:
fig, ax1 = plt.subplots(figsize=(20,10))
graph = sns.countplot(ax=ax1,x='user', hue='activity', data=df)
graph.set_xticklabels(graph.get_xticklabels(),rotation=45)
#for container in ax1.containers:
    #ax1.bar_label(container)

In [24]:
activity = (df['activity'].unique()).tolist()
#print(activity[5])

In [25]:
def plot_activity(activity, user):
    data = df[(df['user'] == user)&(df['activity'] == activity)][:200]
    plt.figure(figsize = (10, 6))
    sns.lineplot(y = 'x-axis', x = 'timestamp', data = data)
    sns.lineplot(y = 'y-axis', x = 'timestamp', data = data)
    sns.lineplot(y = 'z-axis', x = 'timestamp', data = data)
    plt.legend(['x-axis', 'y-axis', 'z-axis'])
    plt.ylabel(activity)
    plt.title(activity, fontsize = 15)
    plt.show()

In [26]:

for act in activity:
    plot_activity(act,20)

        

Filter length prediction: 
Mitra's book Handbook for Digital Signal Processing quotes Kaiser with a simple estimate:

N = [-20*log10(sqrt(dp*ds)) - 13]/[14.6(ws - wp)/2pi]

where dp and ds are the passband ripple peak and stopband ripple peak 
values, ws and wp are the stopband and passband edges - which are normalized 
to 2pi



Sampling rate: 20Hz

In [27]:
#len1 = len(df[df['user'] < 27])
#len2 = len(df)

#print(len1/len2)


In [28]:
#df_train = df[df['user'] < 27]

In [29]:
#df_train.info()

In [30]:
# create the magnitude column
import math 
def get_magnitude(x_axis, y_axis, z_axis):
    return math.sqrt(x_axis**2 + y_axis**2 + z_axis**2)

df['mag'] = df[['x-axis','y-axis','z-axis']].apply(lambda x: get_magnitude(*x), axis=1)

In [31]:
# encoding activity column
# use label encoding technique to encode 
df["activity"] = df["activity"].astype('category')
df["activity_enc"] = df["activity"].cat.codes
#df.activity.cat.codes
#convert back to string: df.activity.cat.categories[df.activity.cat.codes]
# the encoding rule is as follows:
dict( enumerate(df['activity'].cat.categories ) )

In [32]:
# after encoding, drop the activity column
df.drop(['activity'], axis = 1, inplace=True)
#df.head()

In [33]:
#type(df['x-axis'])

In [34]:
def extract_windows(array, start, window_size):
    sub_wins = []
    
    for i in range(window_size + 1):
        sub_win = array[start+i : start + window_size + i]
        sub_wins.append(np.expand_dims(sub_win, 0))
    
    return np.hstack(sub_wins)

In [35]:
sub_winX = []
sub_winY = []
sub_winZ = []
sub_winMag = []
sub_activity=[]
#data_size = len(df_train)
def extract_windows2(array, data_size, window_size,step_size):
    sub_wins = []
    for i in range (0, data_size + 1 - window_size, step_size):
        
        sub_win = array[i : i + window_size]
        sub_wins.append(np.expand_dims(sub_win, 0))
        
    return (sub_wins)



In [36]:
data_size = len(df)
window_size = 100
step_size =50

sub_winX = extract_windows2(df['x-axis'], data_size = data_size, window_size=window_size, step_size=step_size)
sub_winY = extract_windows2(df['y-axis'], data_size = data_size, window_size=window_size, step_size=step_size)
sub_winZ = extract_windows2(df['z-axis'], data_size = data_size, window_size=window_size, step_size=step_size)
sub_winMag = extract_windows2(df['mag'], data_size = data_size, window_size=window_size, step_size=step_size)
sub_winAct = extract_windows2(df['activity_enc'], data_size = data_size, window_size=window_size, step_size=step_size)

In [37]:
type(sub_winX)

In [38]:
print(sub_winX[0:5])

In [39]:
# labelling each subset of data which has extracted by windows sliding techniques
#sub_winAct.shape


In [40]:
#type (sub_winAct[0])
#print (sub_winAct.shape)

In [41]:
#print (len(sub_winAct))

In [42]:
label_lst = []
for a in sub_winAct: 
    b = a.reshape(-1)
    
    act = np.bincount(b).argmax()
    label_lst.append(act)


y = pd.DataFrame(label_lst, columns = ['activity_enc'])

In [43]:
# Feature extraction for the motion features
# use 'mag' column
# calculate the motion features over  time-windows
# fs = 20 samples/second
# windows size = 100 samples
# windows duration = 100/20 = 5 seconds

In [44]:
X = pd.DataFrame()


In [45]:
# Calculate the statistic features on the magnitude of the acceleration
# we ignore the sign of each axis and just focus on the motion features
# magnitude mean

X['mag_mean'] = pd.Series(sub_winMag).apply(lambda x: x.mean())

In [46]:
#std
X['mag_std'] = pd.Series(sub_winMag).apply(lambda x: x.std())

In [47]:
# max, min
X['mag_max'] = pd.Series(sub_winMag).apply(lambda x: x.max())
X['mag_min'] = pd.Series(sub_winMag).apply(lambda x: x.min())

In [48]:
#energy
X['mag_energy'] = pd.Series(sub_winMag).apply(lambda x: np.sum(x**2)/window_size)

Ref for skewness and kurtosis
https://www.itl.nist.gov/div898/handbook/eda/section3/eda35b.htm#:~:text=Skewness%20is%20a%20measure%20of,relative%20to%20a%20normal%20distribution.

In [49]:
#X['mag_skewness'] = pd.Series(sub_winMag).apply(lambda x: stats.skew(x))

In [50]:
#X['mag_kurtosis'] = pd.Series(sub_winMag).apply(lambda x: stats.kurtosis(x))

In [51]:
# as mag >= 0 --> we just calculate the mean crossing rate
def zcr(arr):
    np_array = np.array(arr)
    #return float("{0:.2f}".format((((np_array[:-1] * np_array[1:]) < 0).sum())/len(arr)))
    zc = np.where(np.diff(np.signbit(np_array)))[0]
    zc = len(zc)/ len(arr)
    return zc

In [52]:
def mcr(arr):
    return zcr(np.array(arr) - np.mean(arr))

In [53]:
X['mag_MCR'] = pd.Series(sub_winMag).apply(lambda x: mcr(x))

In [54]:
#X.shape


In [55]:
#y.shape

In this part, the orientation features are extracted based on the raw data from each axis

In [56]:
def rms(arr):
    np_array = np.array(arr)
    mse = np.sum( (np_array)**2 ) / len(np_array)
    return float("{0:.2f}".format(np.sqrt(mse)))

In [57]:
# std
X['x_std'] = pd.Series(sub_winX).apply(lambda x: x.std())
X['y_std'] = pd.Series(sub_winY).apply(lambda x: x.std())
X['z_std'] = pd.Series(sub_winZ).apply(lambda x: x.std())

In [58]:
# rms
X['x_rms'] = pd.Series(sub_winX).apply(lambda x: rms(x))
X['y_rms'] = pd.Series(sub_winY).apply(lambda x: rms(x))
X['z_rms'] = pd.Series(sub_winZ).apply(lambda x: rms(x))


In [59]:
# zcr
X['x_zcr'] = pd.Series(sub_winX).apply(lambda x: zcr(x))
X['y_zcr'] = pd.Series(sub_winY).apply(lambda x: zcr(x))
X['z_zcr'] = pd.Series(sub_winZ).apply(lambda x: zcr(x))

In [60]:
#X['x_skewness'] = pd.Series(sub_winX).apply(lambda x: stats.skew(x))

In [61]:
X.shape

In [62]:
y.shape

In [63]:
#y.head()

In the previous part, the X, y dataset was prepared with 15 features
We will try with multiple models to predict the user's activity

In [64]:
X.head()

In [65]:
X.describe()